In [2]:
import spacy
import pickle
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from typing import List, Tuple, Dict
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

from typing import List, Tuple, Dict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import operator
import torch
from torch import nn, optim
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss

from torchtext.data.metrics import bleu_score

### constants ###

PARENT_DIR = '/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/'
n_extra_tokens = 4
UNK_IDX = 0 
PADDING_IDX = 1 
BOS_IDX = 2
EOS_IDX = 3 

# !pip install spacy==3.1.1

# import spacy.cli
# spacy.cli.download("de_core_news_lg")
# spacy.cli.download("en_core_web_lg")

# source_nlp = spacy.load("de_core_news_lg")
# target_nlp = spacy.load("en_core_web_lg")

In [3]:
import torch
from google.colab import drive
drive.mount('/content/gdrive') # for google colab. adjust accordingly

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import sys  
sys.path.insert(0, PARENT_DIR)

Mounted at /content/gdrive
True
Tesla P100-PCIE-16GB
cuda


In [4]:
# from load_data import *

from EncoderLSTM import *
from DecoderLSTM import *
from Seq2Seq import *
from train import *

from load_data import load_pickled_dataloaders

# load small dataloaders batchsize = 1
mt_train_ds, mt_val_ds, mt_test_ds, mt_train_dl, mt_val_dl, mt_test_dl = load_pickled_dataloaders()
accum_iter = 32

# load small dataloaders batchsize = 32
# mt_train_ds, mt_val_ds, mt_test_ds, mt_train_dl, mt_val_dl, mt_test_dl = load_pickled_dataloaders(batch1 = False)
# accum_iter = 1

In [5]:
batch = next(iter(mt_train_dl))
len(batch), len(batch[1])

(4, 1)

# Beam Search Generate Function

In [6]:
from queue import PriorityQueue
        
@torch.no_grad()
def generate(self, source, max_steps, eos_idx, bos_idx):
# def generate(self, source: torch.tensor, max_steps: int, eos_idx: int, bos_idx: int) -> torch.tensor:
    hidden = self.encoder(source, generate = True)
    # first token in sequence: end of sentence tag
    seq = [bos_idx]
    
    return self.beam_decode(max_steps, hidden, eos_idx, bos_idx)
    
@torch.no_grad()
def beam_decode(self, max_steps, decoder_hidden, EOS_token, BOS_token,  encoder_output=None) -> torch.tensor:
    # target_tensor = target_tensor.permute(1, 0)
    beam_width = 10
    topk = 1
    decoded_batch = []

    for idx in range(1):
        # first token through decoder: beginning of sentence tag
        decoder_input = torch.Tensor([[BOS_token]]).long().to(device)
        
        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))
        
    
        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            # print('--best node seqs len {} '.format(n.leng))
            decoder_input = n.wordid
            decoder_hidden = n.h
            
            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
            # decoder_output, decoder_hidden, _ = self.decoder(decoder_input, decoder_hidden, encoder_output)
            if decoder_input.dim() == 1:
                decoder_input = decoder_input.unsqueeze(0)
            decoder_output, decoder_hidden = self.decoder(
                decoder_input, hidden_init = decoder_hidden[0].to(device), cell_init = decoder_hidden[1].to(device), generate = True
            )
            # PUT HERE REAL BEAM SEARCH OF TOP
            pred = self.linear(decoder_hidden[0])[0]
            pred = self.logsoft(pred)[0]
            log_prob, indexes = torch.topk(pred, beam_width)
            nextnodes = []
            for new_k in range(beam_width):
                decoded_t = indexes[new_k].view(-1)
                log_p = log_prob[new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]
            utterances.append(utterance)
        
        # print(utterances.item())

    for i in range(len(utterances[0])):
        decoded_batch.append(utterances[0][i].item())
    return decoded_batch
        
    
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward

    def __lt__(self, other):
        return self.leng < other.leng 

    def __gt__(self, other):
        return self.leng > other.leng

@torch.no_grad()
def generate_greedy(self, source: torch.tensor, max_steps: int, eos_idx: int, bos_idx: int) -> torch.tensor:
        # the argument source will be a 1D tensor of size ti
        # you will return a 1D translation tensor with maximium length max_steps
        
        # pass source through encoder
        (h_last, c_last) = self.encoder(source, generate = True)
        
        # first token through decoder: beginning of sentence tag
        current_in = torch.Tensor([[bos_idx]]).long().to(device)
        # first token in sequence: end of sentence tag
        seq = [bos_idx]

        while (len(seq) < max_steps):
            # pass current input and encoder states to decoder
            state_last, (h_last, c_last) = self.decoder(
                current_in, hidden_init = h_last.to(device), cell_init = c_last.to(device), generate = True
            )
            
            # predict next token
            pred = self.linear(h_last)
            logsoftpred = self.logsoft(pred[0])
            idxpred = int((logsoftpred[0][1:].max(0))[1]) + 1
            
            # next token through decoder: prediction
            current_in = torch.Tensor([[idxpred]]).long()
            
            # add token to sequence
            seq.append(idxpred)

            if idxpred == eos_idx: # end if we predict end of sentence or seq too long
                break
            if idxpred == 1:
                break

        return seq

# Develop POS BLEU score function

In [7]:
model = torch.load(PARENT_DIR+"jennas_lstm_model")
candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)
bleu = bleu_score(candidate_text, reference_text)
bleu

0.050201352685689926

In [8]:
mt_train_ds.target_vocab['things']

96

In [9]:
candidate_pos = [[str(mt_train_ds.target_vocab_to_tags[mt_train_ds.target_vocab[token]]) for token in seq] for seq in candidate_text]
reference_pos = [[[str(mt_train_ds.target_vocab_to_tags[mt_train_ds.target_vocab[token]]) for token in seq[0]]] for seq in reference_text]

bleu = bleu_score(candidate_pos, reference_pos)
bleu

0.17771677672863007

In [10]:
def pos_bleu_score(candidate_text, reference_text, n = 4):
  candidate_pos = [[str(mt_train_ds.target_vocab_to_tags[mt_train_ds.target_vocab[token]]) for token in seq] for seq in candidate_text]
  reference_pos = [[[str(mt_train_ds.target_vocab_to_tags[mt_train_ds.target_vocab[token]]) for token in seq[0]]] for seq in reference_text]

  weights = [1/n]*n
  bleu = bleu_score(candidate_pos, reference_pos, max_n = n, weights = weights)
  return bleu

In [11]:
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

(0.17771677672863007, 0.2822387218475342, 0.6504767537117004)

# Compute BLEU and POS BLEU for each best model

# LSTM

In [47]:
model = torch.load(PARENT_DIR+"jennas_lstm_model")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.05186642326907656, 0.09054638619018263, 0.3125915576010793)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18146553655566605, 0.2677433884325732, 0.5344107386634629)

In [ ]:
model_swap = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_swap/epoch18")

## convert model to generate with beam
model_swap.beam_decode = beam_decode.__get__(model_swap)
model_swap.generate = generate.__get__(model_swap)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_swap, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.058401897479022004, 0.0985471215816309, 0.3144359410675113)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18930427994870455, 0.27188573265205596, 0.5476374561710743)

In [ ]:
model_drop = torch.load('/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_drop/epoch13')

## convert model to generate with beam
model_drop.beam_decode = beam_decode.__get__(model_drop)
model_drop.generate = generate.__get__(model_drop)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_drop, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.05969504042592338, 0.09917301162625364, 0.32687987928712375)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18064055198035447, 0.2716074961712909, 0.5560100934185466)

In [ ]:
model_blank = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_blank/epoch18")

## convert model to generate with beam
model_blank.beam_decode = beam_decode.__get__(model_blank)
model_blank.generate = generate.__get__(model_blank)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_blank, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.06649916675028462, 0.10729916196009211, 0.3214629740984589)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18222289955999468, 0.267052881225863, 0.5543660287173434)

In [ ]:
model_smooth = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_smooth/epoch16")

## convert model to generate with beam
model_smooth.beam_decode = beam_decode.__get__(model_smooth)
model_smooth.generate = generate.__get__(model_smooth)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_smooth, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.061991637668422264, 0.10226693669938834, 0.3238870768337572)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18720046842812627, 0.27508726605863, 0.5504515401890175)

In [ ]:
model_smooth_pos = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_smooth_pos/epoch19")

## convert model to generate with beam
model_smooth_pos.beam_decode = beam_decode.__get__(model_smooth_pos)
model_smooth_pos.generate = generate.__get__(model_smooth_pos)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_smooth_pos, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.06256970792471653, 0.09927648951260509, 0.30610235550976234)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18936072592597486, 0.26777306768230036, 0.5126911725998422)

In [ ]:
model_lmsample = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_lmsample/epoch18")

## convert model to generate with beam
model_lmsample.beam_decode = beam_decode.__get__(model_lmsample)
model_lmsample.generate = generate.__get__(model_lmsample)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_lmsample, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.06138598327492758, 0.10092492641368009, 0.32933634098667236)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18550795884018237, 0.27304448254996344, 0.5587791882034457)

In [ ]:
model_lmsample_pos = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_lmsample_pos/epoch18")

## convert model to generate with beam
model_lmsample_pos.beam_decode = beam_decode.__get__(model_lmsample_pos)
model_lmsample_pos.generate = generate.__get__(model_lmsample_pos)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_lmsample_pos, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.05375739733454709, 0.09142529189294984, 0.3054944420239205)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.17235390963152622, 0.2556835061628552, 0.5218224742709604)

In [ ]:
model_soft = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_soft/epoch15")

## convert model to generate with beam
model_soft.beam_decode = beam_decode.__get__(model_soft)
model_soft.generate = generate.__get__(model_soft)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_soft, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

(0.19115391941982188, 0.27853992882290834, 0.556687082023536)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.054090773887471445, 0.09313709338175913, 0.32290361355485714)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.19115391941982188, 0.27853992882290834, 0.556687082023536)

In [ ]:
model_soft_pos = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_soft_pos/epoch18")

## convert model to generate with beam
model_soft_pos.beam_decode = beam_decode.__get__(model_soft_pos)
model_soft_pos.generate = generate.__get__(model_soft_pos)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_soft_pos, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.05927907638060394, 0.10148526919737809, 0.32634767118896585)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.19166162807959153, 0.283017244457995, 0.5713467489206742)

In [ ]:
model_seqmix = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_seqmix/epoch14")

## convert model to generate with beam
model_seqmix.beam_decode = beam_decode.__get__(model_seqmix)
model_seqmix.generate = generate.__get__(model_seqmix)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.06654836828247372, 0.11295041567672272, 0.3504999395723375)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.1988317848825245, 0.29563348684979984, 0.5898339054142923)

In [ ]:
model_seqmix_wob = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_seqmix_wob/epoch18")

## convert model to generate with beam
model_seqmix_wob.beam_decode = beam_decode.__get__(model_seqmix_wob)
model_seqmix_wob.generate = generate.__get__(model_seqmix_wob)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_wob, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.0664365252161928, 0.10633291389226335, 0.3211987356302785)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.19746817059766397, 0.28564877600319355, 0.5504036829146287)

In [ ]:
model_seqmix_pos_wob = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_seqmix_pos_wob/epoch19")

## convert model to generate with beam
model_seqmix_pos_wob.beam_decode = beam_decode.__get__(model_seqmix_pos_wob)
model_seqmix_pos_wob.generate = generate.__get__(model_seqmix_pos_wob)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_pos_wob, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.06786758674454996, 0.10836459532395967, 0.33324584940357216)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.19783994124508542, 0.28663488936329556, 0.5728260516660386)

In [ ]:
model_seqmix_wob = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/jennas_lstm_models_seqmix_len/epoch18")

## convert model to generate with beam
model_seqmix_wob.beam_decode = beam_decode.__get__(model_seqmix_wob)
model_seqmix_wob.generate = generate.__get__(model_seqmix_wob)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_wob, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.06966395088164289, 0.11362576334674798, 0.3516467272718221)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.20412185268702598, 0.2978322570445505, 0.5889102203338146)

In [ ]:
model_seqmix_pos = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/seqmix_pos/epoch17")

## convert model to generate with beam
model_seqmix_pos.beam_decode = beam_decode.__get__(model_seqmix_pos)
model_seqmix_pos.generate = generate.__get__(model_seqmix_pos)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_pos, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.0, 0.006979506951845522, 0.10200321188685596)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.019868994551184127, 0.04360426789482478, 0.16953775762301512)

In [39]:
model_seqmix_dis = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/seq2seq_modelk10dis/epoch17")

## convert model to generate with beam
model_seqmix_dis.beam_decode = beam_decode.__get__(model_seqmix_dis)
model_seqmix_dis.generate = generate.__get__(model_seqmix_dis)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_dis, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [40]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.015559541992843151, 0.031748246401548386, 0.12852835655212402)

In [41]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.0633043572306633, 0.10177557915449142, 0.23403674364089966)

In [ ]:
model_seqmix_sim = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/seq2seq_model_lk10_sim/epoch17")

## convert model to generate with beam
model_seqmix_sim.beam_decode = beam_decode.__get__(model_seqmix_sim)
model_seqmix_sim.generate = generate.__get__(model_seqmix_sim)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_sim, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [70]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.015559541992843151, 0.031748246401548386, 0.12852835655212402)

In [71]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.0633043572306633, 0.10177557915449142, 0.23403674364089966)

In [30]:
model_seqmix_dis = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/seq2seq_model_k10dis_l/epoch17")

## convert model to generate with beam
model_seqmix_dis.beam_decode = beam_decode.__get__(model_seqmix_dis)
model_seqmix_dis.generate = generate.__get__(model_seqmix_dis)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_dis, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [31]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.05038900673389435, 0.08752036839723587, 0.3320695161819458)

In [32]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.199825257062912, 0.31305721402168274, 0.688783586025238)

In [24]:
model_seqmix_sim = torch.load("/content/gdrive/MyDrive/CS287_Research_Project/Jennas_Code/seq2seq_model_lk10_sim/epoch19")

## convert model to generate with beam
model_seqmix_sim.beam_decode = beam_decode.__get__(model_seqmix_sim)
model_seqmix_sim.generate = generate.__get__(model_seqmix_sim)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model_seqmix_sim, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX)

In [25]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.05115880072116852, 0.08553628623485565, 0.31611698865890503)

In [26]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.18321220576763153, 0.2905522882938385, 0.6583696603775024)

# Best Val LSTM

In [13]:
def compute_best_bleu(model_name):
  max_bleu = 0
  epoch = 0
  for i in range(6):
    model = torch.load(PARENT_DIR+model_name+"/epoch1"+str(i))

    ## convert model to generate with beam
    model.beam_decode = beam_decode.__get__(model)
    model.generate = generate.__get__(model)

    val_candidate_text, val_reference_text = translate_corpus(mt_val_ds.target_vocab, mt_test_dl, model, 
                                                      eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                      padding_idx = PADDING_IDX, max_steps = 40)

    bleu = bleu_score(val_candidate_text, val_reference_text)
    if bleu > max_bleu:
      epoch = 13+i
      max_bleu = bleu
      uni_bleu = bleu_score(val_candidate_text, val_reference_text, max_n = 1, weights = [1])

  print(model_name)
  print(epoch)
  print('val bleu')
  print(max_bleu, uni_bleu)
  candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                      eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                      padding_idx = PADDING_IDX, max_steps = 40)

  bleu = bleu_score(candidate_text, reference_text)
  max_bleu = bleu
  uni_bleu = bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])
  pos_bleu = pos_bleu_score(candidate_text, reference_text)
  print('test')
  print('bleu')
  print(max_bleu, uni_bleu)
  print('pos')
  print(pos_bleu)


In [17]:
compute_best_bleu('jennas_lstm_models_none')

jennas_lstm_models_none
15
val bleu
0.06758940730618714 0.33319755560990355
test
bleu
0.05976725962248107 0.32182616137576336
pos
0.1849588429586056


In [ ]:
compute_best_bleu('jennas_lstm_models_swap')

In [ ]:
compute_best_bleu('jennas_lstm_models_drop')

In [ ]:
compute_best_bleu('jennas_lstm_models_blank')

In [21]:
compute_best_bleu('jennas_lstm_models_smooth')

jennas_lstm_models_smooth
17
val bleu
0.059224352777236894 0.3139275254692306
test
bleu
0.05517025201869412 0.30907161485569057
pos
0.16775065490599675


In [20]:
compute_best_bleu('jennas_lstm_models_smooth_pos')

jennas_lstm_models_smooth_pos
16
val bleu
0.07394149289500693 0.33513151506227173
test
bleu
0.05869960379090039 0.3219150553074294
pos
0.20111495558768414


In [ ]:
compute_best_bleu('jennas_lstm_models_lmsample')

model_name
15
bleu
0.06152625788423086 0.3191361646136023
pos
0.18111685134365052


In [ ]:
compute_best_bleu('jennas_lstm_models_lmsample_pos')

In [ ]:
compute_best_bleu('jennas_lstm_models_soft')

model_name
18
bleu
0.06067930788293316 0.33533001043339217
pos
0.19677251073868757


In [ ]:
compute_best_bleu('jennas_lstm_models_soft_pos')

model_name
16
bleu
0.06515304618039133 0.30821512516120636
pos
0.18142520558431718


In [ ]:
compute_best_bleu('jennas_lstm_models_seqmix')

model_name
15
bleu
0.06727568376271593 0.34375980448574744
pos
0.19450738542760745


In [ ]:
compute_best_bleu('jennas_lstm_models_seqmix_wob')

model_name
18
bleu
0.06643654111531945 0.3211987356302785
pos
0.19746817059766397


In [ ]:
compute_best_bleu('jennas_lstm_models_seqmix_pos_wob')

model_name
17
bleu
0.06533798323771503 0.3358403861845176
pos
0.1809950039226685


In [ ]:
compute_best_bleu('jennas_lstm_models_seqmix_len')

model_name
13
bleu
0.0731982067419322 0.34532298072524986
pos
0.2053176150527535


In [ ]:
compute_best_bleu('seqmix_pos')

In [ ]:
compute_best_bleu('seq2seq_model_k10dis_l')

model_name
13
bleu
0.029651420190930367 0.2252124845981598
pos
0.12394943833351135


In [ ]:
compute_best_bleu('seq2seq_model_lk10_sim')

model_name
13
bleu
0.02370024286210537 0.17362304031848907
pos
0.08649761229753494


In [18]:
compute_best_bleu('seq2seq_modelk2sim')

seq2seq_modelk2sim
15
val bleu
0.05255035683512688 0.2685062289237976
test
bleu
0.04895903542637825 0.2749204635620117
pos
0.1536942720413208


In [19]:
compute_best_bleu('seq2seq_modelk2dis')

seq2seq_modelk2dis
15
val bleu
0.04235214367508888 0.24092330038547516
test
bleu
0.04098111391067505 0.23019199073314667
pos
0.12892752885818481


In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.30929093845784833, 0.4270353147961752, 0.7603477089609125)

# Transformers

In [12]:
from seq2seqTF import Seq2SeqTF
from batchTF import Batch
from trainTF import train, translate_corpus
from load_lm import *

In [ ]:
model = torch.load(PARENT_DIR+"jennas_models/epoch17")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.10717805011179062, 0.16746532415444307, 0.47824153573373207)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.2968565518606333, 0.41750752282310866, 0.7565624178218039)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_drop")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.11319195027394231, 0.17460328094013336, 0.4744412654861722)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.29168184786625495, 0.4167523600778249, 0.7557731884645205)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_blank")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.10709090975792937, 0.16672643863875572, 0.46260285101192555)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.2848308394712101, 0.40677184814635337, 0.7509531798399758)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_swap")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.11173393518434319, 0.17115874459469876, 0.47841292789791795)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.2938383097260417, 0.41898481710390373, 0.7694501805135082)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_smooth")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.11510666459948739, 0.18040776209443343, 0.49082285902795997)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.3105655430613626, 0.42821152126652606, 0.7645195631641376)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_smooth_pos")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.12424526418485586, 0.18636679395586547, 0.48575490162708984)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.3042170719996501, 0.42624339872815453, 0.7513357359256432)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_lm")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)


  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.10749772686954863, 0.17002746489229487, 0.47837131661031085)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.30147405886759243, 0.4269020877660688, 0.7736339961273978)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_lm_pos")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)


  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])

BLEU


(0.11799727380275726, 0.17925353348255157, 0.4829113781452179)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.3104216754436493, 0.4268723130226135, 0.7575949430465698)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_soft")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])


BLEU


(0.12005779483647927, 0.18342775963712188, 0.4829751706285104)

In [ ]:
print('POS BLEU')
pos_bleu_score(candidate_text, reference_text), pos_bleu_score(candidate_text, reference_text, n = 3), pos_bleu_score(candidate_text, reference_text, n = 1)

POS BLEU


(0.29383526717535335, 0.4127234526025607, 0.7406272477050072)

In [ ]:
model = torch.load(PARENT_DIR+"jennas_model_soft_pos")

## convert model to generate with beam
model.beam_decode = beam_decode.__get__(model)
model.generate = generate.__get__(model)

candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                  eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                  padding_idx = PADDING_IDX, max_steps = 40)

  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
print('BLEU')
bleu_score(candidate_text, reference_text), bleu_score(candidate_text, reference_text, max_n = 3, weights = [0.33,0.33,0.34]), bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])


BLEU


(0.12381459257454261, 0.18758033968673488, 0.4927392616125486)

# best transformers

In [13]:
from seq2seqTF import Seq2SeqTF
from batchTF import Batch
from trainTF import train, translate_corpus
from load_lm import *

In [39]:
compute_best_bleu('jennas_models')

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

jennas_models
11
val bleu
0.12366710983371873 0.489183161744386


  0%|          | 0/157 [00:00<?, ?it/s]

test
bleu
0.10773777663916705 0.4669582526947419
pos
0.27829835657712065


In [14]:
def compute_best_bleu(model_name):
  max_bleu = 0
  epoch = 0
  for i in range(9):
    model = torch.load(PARENT_DIR+model_name+"/epoch1"+str(i))

    ## convert model to generate with beam
    model.beam_decode = beam_decode.__get__(model)
    model.generate = generate.__get__(model)

    val_candidate_text, val_reference_text = translate_corpus(mt_val_ds.target_vocab, mt_test_dl, model, 
                                                      eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                      padding_idx = PADDING_IDX, max_steps=40)

    bleu = bleu_score(val_candidate_text, val_reference_text)
    if bleu > max_bleu:
      epoch = 10+i
      max_bleu = bleu
      uni_bleu = bleu_score(val_candidate_text, val_reference_text, max_n = 1, weights = [1])

  print(model_name)
  print(epoch)
  print('val bleu')
  print(max_bleu, uni_bleu)
  candidate_text, reference_text = translate_corpus(mt_train_ds.target_vocab, mt_test_dl, model, 
                                                      eos_idx = EOS_IDX, bos_idx = BOS_IDX,
                                                      padding_idx = PADDING_IDX, max_steps=40)

  bleu = bleu_score(candidate_text, reference_text)
  max_bleu = bleu
  uni_bleu = bleu_score(candidate_text, reference_text, max_n = 1, weights = [1])
  pos_bleu = pos_bleu_score(candidate_text, reference_text)
  print('test')
  print('bleu')
  print(max_bleu, uni_bleu)
  print('pos')
  print(pos_bleu)

In [17]:
compute_best_bleu('jennas_models_soft')

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

jennas_models_soft
11
val bleu
0.12722411100231643 0.496708801716859


  0%|          | 0/157 [00:00<?, ?it/s]

test
bleu
0.10863470986199245 0.45616298456828247
pos
0.2847571461235671


In [18]:
compute_best_bleu('jennas_models_soft_pos')

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

jennas_models_soft_pos
13
val bleu
0.12755889866430317 0.49638027011797226


  0%|          | 0/157 [00:00<?, ?it/s]

test
bleu
0.1200548680623749 0.48687605313317134
pos
0.2947070261438964


In [19]:
compute_best_bleu('jennas_models_lm_pos')

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

jennas_models_lm_pos
17
val bleu
0.1309714815302964 0.4861060909555705


  0%|          | 0/157 [00:00<?, ?it/s]

test
bleu
0.12575608704334584 0.4860658291274877
pos
0.3159156056202501
